In [1]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'artifact/examples'))
	print(os.getcwd())
except:
	pass

In [2]:
import os
import configparser
import numpy as np
import logging
import sys
import random

In [3]:
os.getcwd()

'/Users/d022166/SAP/GitRepos/Exoplanets/notebooks'

In [4]:
ex_id = 'deploytest2'

In [7]:
#mod_path = os.path.join(os.path.dirname(os.getcwd())[:os.getcwd().index('examples')], 'src', 'hana_ml_artifact')
mod_path = "/Users/d022166/SAP/GitRepos/HANAML/artifact/src/hana_ml_artifact"  # to adapt to my local env
sys.path.append(mod_path)
#mod_path2 = os.path.join(os.path.dirname(os.getcwd())[:os.getcwd().index('examples')], 'src')
mod_path2 = "/Users/d022166/SAP/GitRepos/HANAML/artifact/src"   # to adapt to my local env
sys.path.append(mod_path2)
from hana_ml import dataframe as hd
from hana_ml.algorithms.pal.trees import RandomForestClassifier
from hana_ml.algorithms.pal.partition import train_test_val_split
from hana_ml_artifact import Generator 


In [8]:
address="zeus.hana.canary.eu-central-1.whitney.dbaas.ondemand.com"
port=21761
user="SYSTEM"
password="Abcd1234"

In [9]:
conn = hd.ConnectionContext("{}".format(address),int("{}".format(port)), "{}".format(user), "{}".format(password),
                           encrypt="true", sslValidateCertificate="false")

In [10]:
# **Cloud Foundry setttings**

api='https://api.cf.sap.hana.ondemand.com' 
org='PAL_DSP'
space='LCM2'
password='Ib2aPwz$'
user='frank.gottfried@sap.com'


#api = settings.get("cloudfoundry","api")
#org = settings.get("cloudfoundry","org")
#space = settings.get("cloudfoundry","space")
#cfuser = settings.get("cloudfoundry","user")
#cfpass = settings.get("cloudfoundry","password", raw=True)

In [11]:
pwd

'/Users/d022166/SAP/GitRepos/Exoplanets/notebooks'

In [12]:
# Enable SQL Trace
conn.sql_tracer.enable_sql_trace(True)

In [13]:
# Enable whether all hitoric sql calls need to be kept or just the last one of a algorithm/function
conn.sql_tracer.enable_trace_history(True)

In [14]:
train_df = conn.table("EXOTRAIN_FFT", schema="EXOPLANETS")
test_df = conn.table("EXOTEST_FFT", schema="EXOPLANETS")

In [15]:
columns = train_df.columns

In [16]:
# remove "LABEL" and "ID" column from LIST 
columns.remove("LABEL")
columns.remove("ID")

In [17]:
# by symmetry of the fourier series we remove half the columns
columns_to_drop = columns[1599 :]

In [18]:
# remove low frequencies
columns_to_drop = columns[0:100] + columns_to_drop
columns_to_keep = [col for col in columns if col not in columns_to_drop]
print(len(columns_to_keep))

1499


In [19]:
train_df = train_df.drop(columns_to_drop)
test_df = test_df.drop(columns_to_drop)

In [20]:
train_df.filter("LABEL=1").count()

5050

In [21]:
rfc = RandomForestClassifier(conn_context=conn,n_estimators=1000,
                              max_features=1499, random_state=2,
                                 split_threshold=0.00001,
                                  calculate_oob=True, 
                             min_samples_leaf=10, thread_ratio=0.8)

In [22]:
rfc.fit(train_df,features= columns_to_keep, key= 'ID', label='LABEL')

RandomForestClassifier


INFO:hana_ml.ml_base:Executing SQL: DROP TABLE "#PAL_RANDOM_FOREST_DATA_TBL_0"
INFO:hana_ml.ml_base:Executing SQL: CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_RANDOM_FOREST_DATA_TBL_0" AS (SELECT "ID", "FREQ.101", "FREQ.102", "FREQ.103", "FREQ.104", "FREQ.105", "FREQ.106", "FREQ.107", "FREQ.108", "FREQ.109", "FREQ.110", "FREQ.111", "FREQ.112", "FREQ.113", "FREQ.114", "FREQ.115", "FREQ.116", "FREQ.117", "FREQ.118", "FREQ.119", "FREQ.120", "FREQ.121", "FREQ.122", "FREQ.123", "FREQ.124", "FREQ.125", "FREQ.126", "FREQ.127", "FREQ.128", "FREQ.129", "FREQ.130", "FREQ.131", "FREQ.132", "FREQ.133", "FREQ.134", "FREQ.135", "FREQ.136", "FREQ.137", "FREQ.138", "FREQ.139", "FREQ.140", "FREQ.141", "FREQ.142", "FREQ.143", "FREQ.144", "FREQ.145", "FREQ.146", "FREQ.147", "FREQ.148", "FREQ.149", "FREQ.150", "FREQ.151", "FREQ.152", "FREQ.153", "FREQ.154", "FREQ.155", "FREQ.156", "FREQ.157", "FREQ.158", "FREQ.159", "FREQ.160", "FREQ.161", "FREQ.162", "FREQ.163", "FREQ.164", "FREQ.165", "FREQ.166", "FREQ.16

INFO:hana_ml.ml_base:Executing SQL: DROP TABLE "#PAL_RANDOM_FOREST_PARAM_TBL_0"
INFO:hana_ml.ml_base:Executing SQL: CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_RANDOM_FOREST_PARAM_TBL_0" (
    "PARAM_NAME" NVARCHAR(5000),
    "INT_VALUE" INTEGER,
    "DOUBLE_VALUE" DOUBLE,
    "STRING_VALUE" NVARCHAR(5000)
)
INFO:hana_ml.ml_base:Calling executemany with SQL: INSERT INTO "#PAL_RANDOM_FOREST_PARAM_TBL_0" VALUES (?, ?, ?, ?)
and values: [('TREES_NUM', 1000, None, None), ('TRY_NUM', 1499, None, None), ('NODE_SIZE', 10, None, None), ('SPLIT_THRESHOLD', None, 1e-05, None), ('CALCULATE_OOB', True, None, None), ('SEED', 2, None, None), ('THREAD_RATIO', None, 0.8, None), ('ALLOW_MISSING_DEPENDENT', True, None, None), ('HAS_ID', True, None, None)]
INFO:hana_ml.ml_base:Executing SQL: DROP TABLE "#PAL_RANDOM_FOREST_MODEL_TBL_0"
INFO:hana_ml.ml_base:Executing SQL: CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_RANDOM_FOREST_MODEL_TBL_0" (
    "ROW_INDEX" INTEGER,
    "TREE_INDEX" INTEGER,
    "MODEL_CONTE

In [23]:
rfc.confusion_matrix_.collect()

,ACTUAL_CLASS,PREDICTED_CLASS,COUNT
0,2,2,0
1,2,1,37
2,1,2,0
3,1,1,5050


In [24]:
outputdir = '/Users/d022166/tmp/artifacts'
jar_location = '/Users/d022166/Downloads/distribution-master/containers/jupyterlab/hanaml-jupyterlab/base-image/utils/mta_archive_builder-1.1.19.jar'
version = '0.0.1'

In [25]:
artifact_generator = Generator(ex_id, version, 'CROSS_SCHEMA_SANDBOX', conn, outputdir)

ERROR:hana_ml_artifact.sql_processor:No sql entries found for algorithm: None and function None


TypeError: 'NoneType' object is not iterable

In [ ]:
conn.sql_tracer.get_sql_trace()

In [ ]:
print(sys.path)

